In [16]:
import sqlite3
import xml.etree.ElementTree as ET

# --- Database connection ---
conn = sqlite3.connect("GeoDataBase")
cur = conn.cursor()

In [17]:
# --- Load DIGGS XML file ---
tree = ET.parse('example DIGGS file.xml')  # replace with actual file path
root = tree.getroot()

# --- Namespace map for DIGGS 2.6 ---
ns = {
    'diggs': 'http://diggsml.org/schemas/2.6',
    'gml': 'http://www.opengis.net/gml/3.2',
    'xlink': 'http://www.w3.org/1999/xlink'
}

In [18]:
# --- Create tables ---
cur.execute('''
    CREATE TABLE IF NOT EXISTS _Client (
        _Client_ID TEXT PRIMARY KEY,
        clientName TEXT
    )
''')

cur.execute('''
    CREATE TABLE IF NOT EXISTS _Project (
        _Project_ID TEXT PRIMARY KEY,
        _Client_ID TEXT,
        projectName TEXT,
        projectNumber TEXT,
        projectCountry TEXT,
        projectState TEXT,
        projectCounty TEXT,
        coordinateDatum TEXT,
        FOREIGN KEY (_Client_ID) REFERENCES _Client (_Client_ID)
    )
''')

# --- Insert Client(s) ---
clients = {}
for role in root.findall('.//diggs:role/diggs:Role', ns):
    role_type = role.find('diggs:rolePerformed', ns)
    if role_type is not None and role_type.text.strip().lower() == "client":
        ba = role.find('diggs:businessAssociate/diggs:BusinessAssociate', ns)
        if ba is not None:
            client_id = ba.attrib.get('{http://www.opengis.net/gml/3.2}id')
            client_name = ba.find('gml:name', ns).text if ba.find('gml:name', ns) is not None else None
            clients[client_id] = client_name
            
            # Execute insert and check if it worked
            try:
                cur.execute('INSERT OR IGNORE INTO _Client (_Client_ID, clientName) VALUES (?, ?)', (client_id, client_name))
                print(f"✅ Client inserted: {client_name} (ID: {client_id})")
            except sqlite3.Error as e:
                print(f"❌ Error inserting client: {e}")

# --- Optional: Log result ---
if not clients:
    print("⚠️ No client found in DIGGS file.")
else:
    print("✅ Clients loaded:", clients)


✅ Client inserted: Ohio DOT (ID: businessAssociate_40297645454187830622578296462_Num_0)
✅ Clients loaded: {'businessAssociate_40297645454187830622578296462_Num_0': 'Ohio DOT'}


In [19]:
# --- Insert Project ---
project = root.find('.//diggs:project/diggs:Project', ns)
if project is not None:
    project_id = project.attrib.get('{http://www.opengis.net/gml/3.2}id')
    project_name = project.find('gml:name', ns).text if project.find('gml:name', ns) is not None else None
    project_desc = project.find('gml:description', ns).text if project.find('gml:description', ns) is not None else None

    # Pick first client if available
    client_id = next(iter(clients), None)

    try:
        cur.execute('''
            INSERT OR IGNORE INTO _Project (_Project_ID, _Client_ID, projectName, projectNumber, projectCountry, projectState, projectCounty, coordinateDatum)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', (project_id, client_id, project_name, None, None, None, None, None))
        
        print(f"✅ Project inserted: {project_name} (ID: {project_id})")
    except sqlite3.Error as e:
        print(f"❌ Error inserting project: {e}")
else:
    print("❌ No <Project> element found in the XML.")

# --- Verify insertions ---
print("\n--- Verification ---")
cur.execute("SELECT COUNT(*) FROM _Client")
client_count = cur.fetchone()[0]
print(f"Total clients in database: {client_count}")

cur.execute("SELECT COUNT(*) FROM _Project") 
project_count = cur.fetchone()[0]
print(f"Total projects in database: {project_count}")

✅ Project inserted: Ohio DOT SPT Completed 2017-01-04 (ID: Project0)

--- Verification ---
Total clients in database: 1
Total projects in database: 1


In [20]:
# --- Commit changes ---
conn.commit()
conn.close()